In [10]:
import torch
import torch.nn as nn

In [25]:
# generate random weight
min_val, max_val = -7, 7

weight = torch.randint(min_val, max_val + 1, (8, 8, 3, 3)).float()
weight

tensor([[[[ 4., -3., -7.],
          [ 0., -6.,  6.],
          [-6.,  1.,  7.]],

         [[-7.,  3., -2.],
          [-1., -5., -1.],
          [ 0.,  1., -4.]],

         [[ 7.,  2.,  4.],
          [-4.,  0.,  6.],
          [ 6.,  7., -7.]],

         [[ 2.,  7., -2.],
          [-4., -5., -3.],
          [ 2., -6., -1.]],

         [[ 2.,  7.,  5.],
          [-2., -3.,  1.],
          [-1.,  6.,  6.]],

         [[ 4., -5.,  0.],
          [-4., -1., -4.],
          [-4., -7., -6.]],

         [[-2., -4.,  5.],
          [-7., -5., -4.],
          [ 2.,  1.,  7.]],

         [[-6.,  3., -2.],
          [-1.,  1., -6.],
          [-6.,  1., -6.]]],


        [[[-3., -3.,  4.],
          [ 3.,  3.,  5.],
          [ 2., -3., -4.]],

         [[-3.,  4.,  4.],
          [-2.,  3., -5.],
          [ 6., -5., -1.]],

         [[ 0.,  6., -3.],
          [ 3., -5., -4.],
          [-2.,  0., -5.]],

         [[ 6.,  4., -1.],
          [ 5.,  0., -5.],
          [-7., -1.,  5.]],

  

In [26]:
weight.size()

torch.Size([8, 8, 3, 3])

In [27]:
W = torch.reshape(weight, (weight.size(0), weight.size(1), -1))
W.size() # 8, 8, 9

torch.Size([8, 8, 9])

In [ ]:
W

In [33]:
bit_precision = 4
file = open('weight_random.txt', 'w') #write to file
file.write('#col0row7[msb-lsb],col0row6[msb-lst],....,col0row0[msb-lst]#\n')
file.write('#col1row7[msb-lsb],col1row6[msb-lst],....,col1row0[msb-lst]#\n')
file.write('#................#\n')
for kij in range(9):
    for i in range(W.size(0)):  
        for j in range(W.size(1)):
            if (W[i, 7-j, kij].item()<0):
                W_bin = '{0:04b}'.format(int(W[i,7-j,kij].item()+2**bit_precision+0.001))
            else:
                W_bin = '{0:04b}'.format(int(W[i,7-j,kij].item()+0.001))
            for k in range(bit_precision):
                file.write(W_bin[k])        
            #file.write(' ')  # for visibility with blank between words, you can use
        file.write('\n')
file.close() #close file  

In [29]:
# generate random activation
min_val, max_val = 0, 15

activation = torch.randint(min_val, max_val + 1, (128, 8, 4, 4)).float()
activation

tensor([[[[ 2.,  4., 14., 13.],
          [15., 14., 15.,  5.],
          [12.,  8., 13., 14.],
          [ 3.,  0., 15., 10.]],

         [[ 9.,  9.,  2.,  2.],
          [ 5., 15., 10., 15.],
          [ 4.,  2., 10.,  1.],
          [ 7.,  5., 10.,  1.]],

         [[ 5., 12.,  2.,  9.],
          [ 8., 13.,  0., 10.],
          [ 7.,  6., 12.,  3.],
          [ 9.,  8., 15., 12.]],

         ...,

         [[ 5.,  7.,  1., 13.],
          [ 7.,  5.,  9.,  7.],
          [ 0., 10., 15., 10.],
          [10.,  5., 11., 14.]],

         [[14., 13.,  0.,  2.],
          [13.,  9., 15.,  7.],
          [14.,  1.,  2.,  7.],
          [ 3.,  5.,  7.,  1.]],

         [[14.,  7.,  9., 15.],
          [ 2.,  0.,  5.,  2.],
          [ 8.,  7.,  2.,  6.],
          [ 6.,  4.,  5.,  7.]]],


        [[[ 7.,  5.,  5.,  1.],
          [ 7., 13., 11., 13.],
          [14., 11.,  6.,  7.],
          [13., 10.,  3.,  8.]],

         [[ 1.,  9., 13.,  0.],
          [ 9., 15., 15.,  4.],
         

In [30]:
activation.size()

torch.Size([128, 8, 4, 4])

In [34]:
activation_pad = torch.zeros(128, 8, 6, 6).cuda()
activation_pad[ : ,  :, 1:5, 1:5] = activation.cuda()

In [35]:
X = activation_pad[0]
X = torch.reshape(X, (X.size(0), -1))
X.size()

torch.Size([8, 36])

In [36]:
bit_precision = 4
file = open('activation_random.txt', 'w') #write to file
file.write('#time0row7[msb-lsb],time0row6[msb-lst],....,time0row0[msb-lst]#\n')
file.write('#time1row7[msb-lsb],time1row6[msb-lst],....,time1row0[msb-lst]#\n')
file.write('#................#\n')

for i in range(X.size(1)):  # time step
    for j in range(X.size(0)): # row #
        X_bin = '{0:04b}'.format(int(X[7-j,i].item()+0.001))
        for k in range(bit_precision):
            file.write(X_bin[k])        
        #file.write(' ')  # for visibility with blank between words, you can use
    file.write('\n')
file.close() #close file 

In [37]:
conv = torch.nn.Conv2d(in_channels = 8, out_channels=8, kernel_size = 3, padding=1, bias = False)
conv.weight = torch.nn.parameter.Parameter(weight)
relu = nn.ReLU()
output = relu(conv(activation))
output

tensor([[[[  0.,   0.,   0.,   0.],
          [ 42.,   0.,   0.,   0.],
          [  0.,   0.,   0.,   0.],
          [  0.,   0.,   0.,   0.]],

         [[  0.,   0.,   0.,   0.],
          [  0.,   0.,   0.,   0.],
          [  0.,   0.,   0.,   0.],
          [  0.,   0., 265., 183.]],

         [[  0.,   0.,   0.,   0.],
          [  0.,   0.,   1.,   0.],
          [  0.,   0.,   0.,   0.],
          [ 29.,  91., 104.,  13.]],

         ...,

         [[430., 360., 276.,  92.],
          [299., 450., 152.,   0.],
          [155.,  20., 190.,   0.],
          [  0.,   0.,   0.,   0.]],

         [[  0.,  48., 124.,   0.],
          [  0.,   0.,   0.,  16.],
          [  0.,   0.,   0.,   0.],
          [  0.,   0.,  90.,   0.]],

         [[  0., 165.,   0.,   0.],
          [  0.,  13.,   0.,  58.],
          [154.,  48., 383.,  94.],
          [  0., 183., 401., 297.]]],


        [[[  0.,   0.,   0.,   0.],
          [  0.,   0.,   0.,   0.],
          [  0.,   0.,   0.,   0.],

In [38]:
output.size()

torch.Size([128, 8, 4, 4])

In [40]:
out = output[0]
out = torch.reshape(out, (out.size(0), -1))
out.size()

torch.Size([8, 16])

In [41]:
bit_precision = 16
file = open('output_random.txt', 'w') #write to file
file.write('#time0col7[msb-lsb],time0col6[msb-lst],....,time0col0[msb-lst]#\n')
file.write('#time1col7[msb-lsb],time1col6[msb-lst],....,time1col0[msb-lst]#\n')
file.write('#................#\n')

for i in range(out.size(1)):
    for j in range(out.size(0)):
        if (out[7-j,i].item()<0):
            O_bin = '{0:016b}'.format(int(out[7-j,i].item()+2**bit_precision+0.001))
        else:
            O_bin = '{0:016b}'.format(int(out[7-j,i].item()+0.001))
        for k in range(bit_precision):
            file.write(O_bin[k])
        #file.write(' ')  # for visibility with blank between words, you can use
    file.write('\n')
file.close()